In [ ]:
import cv2
import os
import time
import threading
from playsound import playsound
import numpy as np
from deepface import DeepFace
from scipy.spatial.distance import cosine

embeddings_dict = np.load("face_embeddings.npy", allow_pickle=True).item()
embeddings = [embeddings_dict[key] for key in embeddings_dict]

threshold = 0.85
max_no_face_time = 20

frame_counter = 0
skip_threshold = 25
display_duration = 1.5

last_detection_time = 0
last_result = None
last_face_info = None
last_max_similarity = 0
status = False
end_counter = 0

cam = cv2.VideoCapture(0)
# cv2.namedWindow("Lock Face System")
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

if not cam.isOpened():
    print("Error: Could not open camera.")
    exit()

def image_embedding(img):
    try:
        embedding = DeepFace.represent(img, model_name="Facenet", enforce_detection=False, align=False)[0]
        face = embedding.get("facial_area", None)
        if face:
            x, y, w, h = face["x"], face["y"], face["w"], face["h"]
            similarities = [1 - cosine(em, embedding["embedding"]) for em in embeddings]
            max_similarity = max(similarities)
            return embedding["embedding"], True, (x, y, w, h), max_similarity
    except:
        pass
    return None, False, None, 0

start_time = time.time()

while True:
    ret, frame = cam.read()
    if not ret:
        break
    
    frame_counter += 1

    if frame_counter % skip_threshold == 0:
        embedding, result, face_info, max_similarity = image_embedding(frame)
        if embedding is not None:
            last_detection_time = time.time()
            last_result = result
            last_face_info = face_info
            last_max_similarity = max_similarity
            print(max_similarity)
    
    if last_result and last_face_info and time.time() - last_detection_time < display_duration:
        x, y, w, h = last_face_info
        similarity_text = f"{last_max_similarity * 100:.2f}%"
        sec = str(max_no_face_time - (time.time() - start_time))[:4]
        text = f"{sec} sec to lock screen!"
        color = (0, 0, 255)

        if last_max_similarity > threshold:
            text = "This is you, welcome!"
            color = (0, 255, 0)
            status = True

        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2, cv2.LINE_AA)
        cv2.putText(frame, similarity_text, (x + 20, y - 35), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2, cv2.LINE_AA)
    
    cv2.imshow("Face Lock System", frame)

    if status:
        end_counter += 1
        if end_counter > 30:
            threading.Thread(target=playsound, args=("voices/me.mp3",), daemon=True).start()
            break

    if time.time() - start_time > max_no_face_time:
        threading.Thread(target=playsound, args=("voices/not_me.mp3",), daemon=True).start()
        time.sleep(2)
        print("No authorized face detected for too long! Locking system.")
        os.system("rundll32.exe user32.dll,LockWorkStation")
        break

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()


0.8158497731751028
0.7236482859975681
0.5448210900484874
0.6452457340053093
0.7129436171156013
0.7058567260845139
0.768107099938033
0.8082153479855113
0.8608032958773865
0.8706571334155229



    Error 263 for command:
        close voices/me.mp3
    The specified device is not open or is not recognized by MCI.
Failed to close the file: voices/me.mp3
